In [ ]:
# 1 ######################
# Create python environment
# !python -m venv yolo
# !pip install torch numpy notebook ipykernel opencv-python
# !python3 -m ipykernel install --name "yolo" --user
# !sudo apt-get install vlc
# 2 ######################
# !pip install ultralytics
# or
# !conda create -n yolo
# !conda activate yolo
# !conda install -c conda-forge ultralytics
# 2 ######################
# Download yolo11 detection from here: https://docs.ultralytics.com/models/yolo11/#supported-tasks-and-modes
# !wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt
# 3 ######################
# Download bean data
# 3.1 Go to https://universe.roboflow.com/test-fdsxz/beans-dijdm/dataset/1
# 3.2 Login 
# 3.3 Download txt for yolo11
# 4 ######################
# Train

## Command line code


### 1. Setup Environment

```
python -m venv yolo
source yolo/bin/activate
pip install torch numpy notebook ipykernel opencv-python numpy ultralytics
python3 -m ipykernel install --name "yolo" --user
sudo apt-get install vlc
wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt
```

### 2. Images with Roboflow

- Go to: https://universe.roboflow.com/test-fdsxz/beans-dijdm/dataset/1
- Log in and click "download YOLOv11"
  - Download dataset
  - YOLOv11 format
  - Download zip to computer
- Decompress with `unzip <file.zip>`
- `cd` into folder
  
### 3. Train YOLO

- Train YOLO with `yolo detect train data=data.yaml model=yolo11n.pt epochs=100 imgsz=640`
- `cp runs/detect/train3/weights/best.pt ~/`

## Executable code: Single Image

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLO model
model = YOLO("best.pt")

# Path to input video
video_path = r"1135446.mp4"

# Open video stream
cap = cv2.VideoCapture(video_path)

# Read the first frame
success, img = cap.read()

if success:
    # Process the frame
    result_img, _ = predict_and_detect(model, img, classes=[], conf=0.5)
    
    # Save the processed image
    output_image_filename = "output_image.jpg"
    cv2.imwrite(output_image_filename, result_img)
    print(f"Saved: {output_image_filename}")
else:
    print("Failed to read the video file.")

# Release resources
cap.release()
cv2.destroyAllWindows()


## Executable code: Video

In [ ]:
#!pip install opencv-python
import cv2
from ultralytics import YOLO

#OVERVIEW: The yolo model detects objects in a single frame, these detections are used to draw the actual boxes on top of the input frame
#          and write them to a new mp4 file. Rinse and repeat for the entire input video

#Branches yolos predict method based on weather classes are provided
def predict(chosen_model, img, classes=[], conf=0.5):
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    #results are a list of attributes like boxes, masks, etc https://docs.ultralytics.com/modes/predict/#working-with-results
    return results

def predict_and_detect(chosen_model, img, classes=[], conf=0.5, rectangle_thickness=2, text_thickness=1):
    results = predict(chosen_model, img, classes, conf=conf)
    #count variable keeps track of total objects detected by the model (optional functionality)
    count=0
    
    #for each individual detection, draw a rectangle bounding box and title of the detected object
    for result in results:
        for box in result.boxes:

            #rectangle method needs coordinates of the two corners of the rectangle you need drawn
            #https://www.geeksforgeeks.org/python-opencv-cv2-rectangle-method/
            #box.xyxy holds pixel coordinates in a 2d list and are used as points to draw the rectangle
            cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                          (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 0), rectangle_thickness)
            
            #Create text above the generated box that prints the class name
            cv2.putText(img, f"{result.names[int(box.cls[0])]}",
                        (int(box.xyxy[0][0]), int(box.xyxy[0][1]) - 10),
                        cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), text_thickness)
            count += 1

    #creates box and text in output image displaying the total cow count
    cv2.rectangle(img, (10, 5), (165, 50), (0,0,0), -1)
    cv2.putText(img, f"BEANS: {count}", (10,35), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

    #return edited frame (img) containing bounding boxes
    return img, results

# defining function for creating a writer (for mp4 videos)
def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #mp4 file format
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    #return writer has .write method to "write" individual frames of new video with bounding boxes, based on the info outlined above
    return writer

model = YOLO("best.pt")

output_filename = "out.mp4"

video_path = r"1135446.mp4"

#cv2 video stream to run the model against
cap = cv2.VideoCapture(video_path)
writer = create_video_writer(cap, output_filename)

while True:
    success, img = cap.read()
    
    #logic to end loop when video finishes
    if not success:
        break

    #get prcocessed frame and write it into output video file
    result_img, _ = predict_and_detect(model, img, classes=[], conf=0.5)
    writer.write(result_img)

    #show live progress of video writing
    cv2.imshow("Image", result_img)
    
    cv2.waitKey(1)
writer.release()